<a href="https://colab.research.google.com/github/patrickctrf/projeto-ia376/blob/e1gr/E3/notebooks/IA376L_S%C3%ADntese_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Parte 1  - Download dos arquivos necessários.

In [ ]:
!gdown 1wf6hJuC3zcxPP6B_uPWgI--L8c68qskB #primers
!7z e primers.7z -o/content/primers

!rm primers.7z

!mkdir output

!gdown 1JqCEMUylvNZTNPiMUjtoKMJnXaS5AQtH #mid to csv
!gdown 1D-Meh47KU90da7nVVepa_Am1z2c8A0Zm #csv to mid

##Fazendo Mount do Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Instalando o Numpy específico (1.19)

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.19

##Instalando dependências

In [ ]:
!gsutil -q -m cp -r gs://magentadata/models/music_transformer/primers/* /content/
!gsutil -q -m cp gs://magentadata/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /content/
!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install -q 'tensorflow-datasets < 4.0.0'
!pip install -q google-cloud magenta pyfluidsynth

##Instalando Magenta

In [ ]:
!curl https://raw.githubusercontent.com/tensorflow/magenta/main/magenta/tools/magenta-install.sh > /tmp/magenta-install.sh
!bash /tmp/magenta-install.sh

#Parte 2

##Instalando as Bibliotecas necessárias

In [ ]:
%tensorflow_version 1.x

import numpy as np
import os
import tensorflow.compat.v1 as tf

from google.colab import files

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.utils import decoding
from tensor2tensor.utils import trainer_lib

from magenta.models.score2perf import score2perf
import note_seq

tf.disable_v2_behavior()

##Definições de variáveis

In [ ]:
SF2_PATH = '/content/Yamaha-C5-Salamander-JNv5.1.sf2'
SAMPLE_RATE = 16000

def decode(ids, encoder):
  ids = list(ids)
  if text_encoder.EOS_ID in ids:
    ids = ids[:ids.index(text_encoder.EOS_ID)]
  return encoder.decode(ids)

##Instalando o transformer pré-treinado para sequenciar uma entrada primer

In [ ]:
model_name = 'transformer'
hparams_set = 'transformer_tpu'
ckpt_path = 'gs://magentadata/models/music_transformer/checkpoints/unconditional_model_16.ckpt'

class PianoPerformanceLanguageModelProblem(score2perf.Score2PerfProblem):
  @property
  def add_eos_symbol(self):
    return True

problem = PianoPerformanceLanguageModelProblem()
unconditional_encoders = problem.get_feature_encoders()

hparams = trainer_lib.create_hparams(hparams_set=hparams_set)
trainer_lib.add_problem_hparams(hparams, problem)
hparams.num_hidden_layers = 16
hparams.sampling_method = 'random'

decode_hparams = decoding.decode_hparams()
decode_hparams.alpha = 0.0
decode_hparams.beam_size = 1

run_config = trainer_lib.create_run_config(hparams)
estimator = trainer_lib.create_estimator(
    model_name, hparams, run_config,
    decode_hparams=decode_hparams)

def input_generator():
  global targets
  global decode_length
  while True:
    yield {
        'targets': np.array([targets], dtype=np.int32),
        'decode_length': np.array(decode_length, dtype=np.int32)
    }

targets = []
decode_length = 0

input_fn = decoding.make_input_fn_from_generator(input_generator())
unconditional_samples = estimator.predict(
    input_fn, checkpoint_path=ckpt_path)

_ = next(unconditional_samples)

##Formatando a entrada primer

In [ ]:
primer_ns = note_seq.midi_file_to_note_sequence('/content/primers/Game_of_Thrones_Melody_Polyphonic.mid')
primer_ns = note_seq.apply_sustain_control_changes(primer_ns)

max_primer_seconds = 20 
if primer_ns.total_time > max_primer_seconds:
  print('Primer is longer than %d seconds, truncating.' % max_primer_seconds)
  primer_ns = note_seq.extract_subsequence(
      primer_ns, 0, max_primer_seconds)

if any(note.is_drum for note in primer_ns.notes):
  print('Primer contains drums; they will be removed.')
  notes = [note for note in primer_ns.notes if not note.is_drum]
  del primer_ns.notes[:]
  primer_ns.notes.extend(notes)

for note in primer_ns.notes:
  note.instrument = 1
  note.program = 0

##Fazendo o decode do áudio

In [ ]:
targets = unconditional_encoders['targets'].encode_note_sequence(
    primer_ns)

targets = targets[:-1]

decode_length = max(0, 4096 - len(targets))
if len(targets) >= 4096:
  print('Primer has more events than maximum sequence length; nothing will be generated.')

sample_ids = next(unconditional_samples)['outputs']

midi_filename = decode(
    sample_ids,
    encoder=unconditional_encoders['targets'])
ns = note_seq.midi_file_to_note_sequence(midi_filename)

continuation_ns = note_seq.concatenate_sequences([primer_ns, ns])

note_seq.sequence_proto_to_midi_file(
    continuation_ns, '/tmp/continuation.mid')

##Instalando o transformer pré-treinado para fazer acompanhamento do arquivo midi gerado anteriormente

In [ ]:
model_name = 'transformer'
hparams_set = 'transformer_tpu'
ckpt_path = 'gs://magentadata/models/music_transformer/checkpoints/melody_conditioned_model_16.ckpt'

class MelodyToPianoPerformanceProblem(score2perf.AbsoluteMelody2PerfProblem):
  @property
  def add_eos_symbol(self):
    return True

problem = MelodyToPianoPerformanceProblem()
melody_conditioned_encoders = problem.get_feature_encoders()

hparams = trainer_lib.create_hparams(hparams_set=hparams_set)
trainer_lib.add_problem_hparams(hparams, problem)
hparams.num_hidden_layers = 16
hparams.sampling_method = 'random'

decode_hparams = decoding.decode_hparams()
decode_hparams.alpha = 0.0
decode_hparams.beam_size = 1

run_config = trainer_lib.create_run_config(hparams)
estimator = trainer_lib.create_estimator(
    model_name, hparams, run_config,
    decode_hparams=decode_hparams)

inputs = []
decode_length = 0

def input_generator():
  global inputs
  while True:
    yield {
        'inputs': np.array([[inputs]], dtype=np.int32),
        'targets': np.zeros([1, 0], dtype=np.int32),
        'decode_length': np.array(decode_length, dtype=np.int32)
    }

input_fn = decoding.make_input_fn_from_generator(input_generator())
melody_conditioned_samples = estimator.predict(
    input_fn, checkpoint_path=ckpt_path)

_ = next(melody_conditioned_samples)

##Formatando a entrada midi

In [ ]:
melody_ns = note_seq.midi_to_note_sequence('/content/primers/Game_of_Thrones_Melody_Polyphonic.mid')
melody_instrument = note_seq.infer_melody_for_sequence(melody_ns)
notes = [note for note in melody_ns.notes
        if note.instrument == melody_instrument]
del melody_ns.notes[:]
melody_ns.notes.extend(
    sorted(notes, key=lambda note: note.start_time))
for i in range(len(melody_ns.notes) - 1):
melody_ns.notes[i].end_time = melody_ns.notes[i + 1].start_time
inputs = melody_conditioned_encoders['inputs'].encode_note_sequence(
    melody_ns)
note_seq.plot_sequence(melody_ns)

##Fazendo o decode do áudio

In [ ]:
# Generate sample events.
decode_length = 4096
sample_ids = next(melody_conditioned_samples)['outputs']

# Decode to NoteSequence.
midi_filename = decode(
    sample_ids,
    encoder=melody_conditioned_encoders['targets'])
accompaniment_ns = note_seq.midi_file_to_note_sequence(midi_filename)

note_seq.sequence_proto_to_midi_file(
    accompaniment_ns, '/tmp/accompaniment.mid')